<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

### 1. 位宽推断的使用限制 
关于没有位宽声明的getWidth用法
```scala
  val c = UInt()
  c.getWidth   // X Failed !
  c:= a + b
  c.getWidth   // is ok, because c width already inferred by "c := a + b"
```
另外一个例子， setAll 函数需要getWidth方法
```scala
 val c = out UInt() 
 when(clc){c.setAll }.otherwise(c := a * b)  //failed
 when(clc){c := a * b }.otherwise(c.setAll)  //It's OK
```

In [ ]:
class T2  extends Component{
    val a,b = in UInt(8 bits)
    val clc= in Bool()
    val c = out UInt() 
//  when(clc){c.setAll }.otherwise(c := a * b)  //failed   
  when(clc){c := a * b }.otherwise(c.setAll)  //It's OK
}
showRtl(new T2)

### 2. asBool问题 
对于1bit的数据操作，第一种和后面两个方法是有区别的，请注意
- a.asBits  
- a.lsb     
- a(0) 
```scala
val a= Bits(1 bits)
a.asBits //返回一个新的Bool，将 tmp := a.lsb
a.lsb    //直接操作a(0) bit
a(0)     //
```

In [ ]:
class T1  extends Component{  
  val sel = in Bool()
  val a = Reg(Bits(1 bits)) init 0 
  when(sel){a.asBool.set()} //generate verilog beyond your expectations
}
showRtl(new T1)  

In [ ]:
class T2  extends Component{  
  val sel = in Bool()
  val a = Reg(Bits(1 bits)) init 0 
  when(sel){a(0).set()} 
  // when(sel){a.lsb.set()}  //also Ok
}
showRtl(new T2) 

## 3. val def in Trait
**Attation** val cause Error 

In [ ]:
trait PRNBase {
  val size: Int 

  val Mask = (1 << size) - 1
  val Msb  = (1 << (size - 1))
}

object GPS extends PRNBase{
    val size = 1023
}

object BD extends PRNBase{
    val size = 2046
}
BD.Mask toHexString // return 0

In [ ]:
trait PRNBase {
  val size: Int 

  def Mask = (1 << size) - 1
  def Msb  = (1 << (size - 1))
}
object BD extends PRNBase{
    val size = 11
}
BD.Mask toHexString

### Watch Dog


In [ ]:
class T2  extends Component{ 
    val a = out Bits(9 bit)
    a := 133
    class Dog{}
    val xiaogou = new Dog

}
showRtl(new T2) 

In [ ]:
class T2  extends Component{ 
   class Dog{
      def genTimer(n: Int) = {
         val timer = Reg(UInt(n bits)) init 0 
         val clearTimer = in Bool()
         when(clearTimer){
            timer init 0
         }.otherwise {
            timer := timer + 1
         }
         (clearTimer,timer)
      }
   }
   val xiaogou = new Dog
   val (weigou,timer) = xiaogou.genTimer(8)
} 
showRtl(new T2) 

In [ ]:
 class Dog {
      def genTimer(n: Int) = {
         val timer = Reg(UInt(n bits)) init 0 
         val clearTimer = in Bool()
         when(clearTimer){
            timer init 0
         }.otherwise {
            timer := timer + 1
         }
         (clearTimer,timer)
      }
   }

why not?

In [ ]:
class T2  extends Component{ 
   val xiaogou =  new Dog
   val (weigou,timer) = xiaogou.genTimer(8)
} 
showRtl(new T2) 

In [ ]:
class T2  extends Component{  
   val (weigou,timer) = (new Dog).genTimer(8)
} 
showRtl(new T2) 

In [ ]:
(7 downto 2)
implicit class Intexpand(x: Int) {    
    def ~(y: Int) = (x downto  y)
    def ↓(y: Int) = (x downto  y)
    def ↑(y: Int) = (x to  y)
    def <->(y: Int) = y match{
        case a if a>x => (x to y)
        case _ => (x downto y)
    }
     
}
 (new Intexpand(9)).~(2)
 
(8 downto 2)
(8 ~ 2)
(2 ~ 9 ) 
(8 to 2)
 class T2  extends Component{ 
   val a =  Bits(10 bits)
   val b =  a(8 ↓ 2)
   val c =  a(1 ↑ 5)
   val c0 =  a(2 <-> 6)
   val c1 =  a(9 <-> 1)
} 
showRtl(new T2) 

In [ ]:
class TopLevel extends Component {
      val io = new Bundle {
         val ready = in Bool
         val valid = out Bool
       }
      val valid = RegInit(False)

      when(io.ready){
        valid := False
      }
      io.valid <> valid
      // some logic

      import spinal.core.GenerationFlags._
      import spinal.core.Formal._

       GenerationFlags.formal{
        when(initstate()) {
          assume(clockDomain.isResetActive === True)
          assume(io.ready === False)
        }.otherwise {
          assert(!(valid.fall && !io.ready))
        }
      }
}
showRtl(new TopLevel)

In [ ]:
class Ram_1w_1r(wordWidth: Int, wordCount: Int) extends BlackBox {

  // SpinalHDL will look at Generic classes to get attributes which
  // should be used ad VHDL gererics / Verilog parameter
  // You can use String Int Double Boolean and all SpinalHDL base types
  // as generic value
  val generic = new Generic {
    val wordCount = Ram_1w_1r.this.wordCount
    val wordWidth = Ram_1w_1r.this.wordWidth
  }

  // Define io of the VHDL entiry / Verilog module
  val io = new Bundle {
    val clk = in Bool
    val wr = new Bundle {
      val en   = in Bool
      val addr = in UInt (log2Up(wordCount) bit)
      val data = in Bits (wordWidth bit)
    }
    val rd = new Bundle {
      val en   = in Bool
      val addr = in UInt (log2Up(wordCount) bit)
      val data = out Bits (wordWidth bit)
    }
  }

  //Map the current clock domain to the io.clk pin
  mapClockDomain(clock=io.clk)
}
showRtl(new Ram_1w_1r)

In [ ]:
class Taps(val taps: Array[Int]) {
  def mask(): Int = {
    taps.map(x => 1<<(x-1)).reduce((x,y) => x + y)
  }
}

object Taps{
  def apply(args: Int*) = {
    new Taps(args.toArray)
  }
}

In [ ]:
Taps(1,7,8,9,10,11).mask toHexString

In [ ]:
Taps(1,2,3,4,5,8,9,11).mask toHexString

In [ ]:
Taps(3,10).mask toHexString

In [ ]:
Taps(2,3,6,8,9,10).mask toHexString

In [ ]:
class T1 extends Component{
    val a = in SInt(8 bits)
    val b = a |>> 3
}
showRtl(new T1)

In [ ]:
showRtl( new StreamFifo(
    dataType = Bits(32 bits),
    depth = 32
  ))

In [ ]:
showRtl(new StreamFifoCC(
    dataType = Bits(32 bits),
    depth = 32,
    pushClock = ClockDomain.external("clkA"),
    popClock = ClockDomain.external("clkB")
  ))

In [ ]:
implicit class UIntexpand(x: UInt) = {
   def apply(n: Int): Bool = {
     VectorInit(x)(n)
   }
}
a(0) := False 应该就可以了